In [ ]:
#pip install vtk
#pip install epyseg

## TOP_tst_vtkvrml_importer_working.py

In [1]:
import vtk
import numpy as np
from vtkmodules.util.numpy_support import vtk_to_numpy
import random
from epyseg.img import Img, int24_to_RGB

'''
code by Benoit Aigouy
code to voxelize mesh files (a bit slow)
'''

def get_forbidden_colors_int24():
    # return [r_g_b_to_rgb(0, 0, 0), r_g_b_to_rgb(255, 255, 255)]
    return [0,0xFFFFFF]

def r_g_b_to_rgb(r, g, b):
    RGB = (r << 16) + (g << 8) + b
    return RGB

def get_unique_random_color_int24(forbidden_colors=None, seed_random=None, assign_new_col_to_forbidden=False):
    if seed_random is not None:
        random.seed(seed_random)
    r = random.getrandbits(8)
    b = random.getrandbits(8)
    g = random.getrandbits(8)
    color = r_g_b_to_rgb(r, g, b)
    # color = (r, g, b)
    if forbidden_colors is not None:
        if color in forbidden_colors:
            while color in forbidden_colors:
                color = get_unique_random_color_int24(forbidden_colors=forbidden_colors)
        if assign_new_col_to_forbidden:
            forbidden_colors.append(color)
    return color

def mesh2Image(raw_3D_image_file, mesh_file):
    ren1 = vtk.vtkRenderer()
    renWin = vtk.vtkRenderWindow()
    renWin.AddRenderer(ren1)
    importer = vtk.vtkVRMLImporter()
    importer.SetRenderWindow(renWin)

    raw_stack = Img(raw_3D_image_file)
    final_binarised_image_size = raw_stack.shape
    # remove color channel
    if raw_stack.has_c():
        final_binarised_image_size = final_binarised_image_size[:-1]
        # print('final_binarised_image_size', final_binarised_image_size)

    importer.SetFileName(mesh_file)
    importer.Read()

    actors = importer.GetRenderer().GetActors()

    actors.InitTraversal()

    polydatas = []

    for iii in range(actors.GetNumberOfItems() - 1):
        polydata = vtk.vtkPolyData.SafeDownCast(actors.GetNextActor().GetMapper().GetInput())
        polydatas.append(polydata)

    image = mesh2Volume2(polydatas,
                         final_binarised_image_size=final_binarised_image_size)  # real image spacing # if I do so I have just

    return image

# based on https://kitware.github.io/vtk-examples/site/Python/PolyData/PolyDataToImageDataStencil/ and on https://web.archive.org/web/20201128015112/https://lorensen.github.io/VTKExamples/site/Cxx/PolyData/PolyDataToImageData/
def mesh2Volume2(pds, spacing=(1, 1, 1), final_binarised_image_size=(40,1024,1024), bounds=None):

    if bounds is None:
        min_x = min_y = min_z= 100000
        max_x = max_y = max_z= 0
        for polydata in pds:
            # print(polydata.GetBounds())
            bounds = polydata.GetBounds()
            min_x = min(min_x, bounds[0])
            max_x = max(max_x, bounds[1])
            min_y = min(min_y, bounds[2])
            max_y = max(max_y, bounds[3])
            min_z = min(min_z, bounds[4])
            max_z = max(max_z, bounds[5])
        bounds = [min_x, max_x, min_y, max_y, min_z, max_z]


    whiteImage = vtk.vtkImageData()
    seed = 1  # always start tracking with same seed to have roughly the same color
    assigned_ids = get_forbidden_colors_int24()

    whiteImage.SetSpacing(spacing)

    spacing = [(bounds[1]-bounds[0])/final_binarised_image_size[2], (bounds[3]-bounds[2])/final_binarised_image_size[1], (bounds[5]-bounds[4])/final_binarised_image_size[0]]
    bounds = [0, bounds[1], 0, bounds[3], 0, bounds[5]] # necessaire pr avoir l'image desiree en fait --> pas sur de comprendre mais ok et ensuite faut retailler l'image --> cropper la difference

    # compute dimensions
    dim = [0, 0, 0]
    for i in [0, 1, 2]:
        dim[i] = int(np.ceil((bounds[i * 2 + 1] - bounds[i * 2]) / spacing[i]))

    # print('dim',dim) # bug is here --> too small --> dim [604, 641, 83] for spacing (0.01,0.01,0.1) --> just get real spacing in fact!!!
    whiteImage.SetDimensions(dim)
    whiteImage.SetExtent(0, dim[0] - 1, 0, dim[1] - 1, 0, dim[2] - 1)

    whiteImage.AllocateScalars(vtk.VTK_UNSIGNED_CHAR, 1)


    dims = whiteImage.GetDimensions()
    merge_image =  np.zeros((dims[2]*dims[1]*dims[0]))
    merge_image =  merge_image.reshape(dims[2], dims[1], dims[0])
    merge_image= merge_image.transpose(2, 1, 0)

    # make image entirely white or any color I want
    # fill the image with foreground voxels:
    # in fact unused largely except for the size
    inval = 0
    count = whiteImage.GetNumberOfPoints()
    for i in range(count):
        whiteImage.GetPointData().GetScalars().SetTuple1(i, inval)

    # there are probably smarter ways of doing this but it works so ok for now
    for pd in pds:
        bounds2 = pd.GetBounds()
        origin = [0, 0, 0]
        origin[0] = bounds2[0] + spacing[0] / 2
        origin[1] = bounds2[2] + spacing[1] / 2
        origin[2] = bounds2[4] + spacing[2] / 2

        # polygonal data --> image stencil:
        pol2stenc = vtk.vtkPolyDataToImageStencil()
        pol2stenc.SetInputData(pd)
        pol2stenc.SetOutputSpacing(spacing)
        pol2stenc.SetOutputWholeExtent(whiteImage.GetExtent())
        pol2stenc.Update()

        # outval = 0
        imgstenc = vtk.vtkImageStencilToImage()
        imgstenc.SetInputData(whiteImage)
        imgstenc.SetInputConnection(pol2stenc.GetOutputPort())
        # imgstenc.SetOutputOrigin(origin) # do not do that otherwise all centered on the same position
        imgstenc.SetOutsideValue(0)
        imgstenc.SetInsideValue(255)
        imgstenc.Update()

        dims = whiteImage.GetDimensions()
        sc = imgstenc.GetOutput().GetPointData().GetScalars()
        a = vtk_to_numpy(sc)
        a = a.reshape(dims[2], dims[1], dims[0])
        a = a.transpose(2, 1, 0)

        # random color image
        new_col = get_unique_random_color_int24(forbidden_colors=assigned_ids, assign_new_col_to_forbidden=True)
        merge_image[a!=0]=new_col

    merge_image = np.rot90(merge_image,k=3) # needed to get the proper orientation
    merge_image = np.fliplr(merge_image)
    merge_image = merge_image[merge_image.shape[0] - final_binarised_image_size[1]:, merge_image.shape[1] - final_binarised_image_size[2]:, :].astype(np.int32)
    merge_image = np.moveaxis(merge_image, -1, 0)
    merge_image = int24_to_RGB(merge_image)

    return merge_image

if __name__ == "__main__":
    # '/D/Sample_images/clara_tests_3D_mesh/E9 WT GM130V actinR 4.lsm'
    #
    # "/D/Sample_images/clara_tests_3D_mesh/E9 WT GM130V actinR 4_mask_v1_PHALLOIDIN.wrl"
    # "/D/Sample_images/clara_tests_3D_mesh/E9 WT GM130V actinR 4_mask_v1_GOLGI.wrl"
    # "/D/Sample_images/clara_tests_3D_mesh/E9 WT GM130V actinR 4_mask_v1_NUCLEUS.wrl"
    img = mesh2Image('E9 WT GM130V actinR 2.lsm', 'E9 WT GM130V actinR2_mask_PHALLOIDIN.wrl')

    Img(img, dimensions='dhwc').save('test.tif') #, mode='raw'

C:\Users\guijarro\AppData\Roaming\Python\Python37\site-packages\tifffile\tifffile.py:1527: UserWarning: TiffWriter: writing zero size array to nonconformant TIFF
  UserWarning,


## tst_3D analysis_image.py

In [2]:
%%time
import numpy as np
from epyseg.img import Img, RGB_to_int24, int24_to_RGB
from skimage.measure import regionprops
from skimage.measure import label
import matplotlib.pyplot as plt
from skimage import draw
import os
#import get_forbidden_colors_int24
#import get_unique_random_color_int24 
#import mesh2Image
import csv

__DEBUG__ = False

'''
code by Benoit Aigouy
code to connect first encountered nucleus to first encountered golgi for every cell segmented by tissue analyzer
'''

path_to_TA_2D_analyzed_file = 'E9 WT GM130V actinR 2_StackFocuser.tif'
path_to_corresponding_3D_image = 'E9 WT GM130V actinR 2.lsm'
filename_without_ext = os.path.splitext(path_to_TA_2D_analyzed_file)[0]
golgi_mesh_file = 'E9 WT GM130V actinR 2_mask_GOLGI.wrl'
nucleus_mesh_file = 'E9 WT GM130V actinR 2_mask_NUCLEUS.wrl'
# convert mesh to voxels

# very slow
print('converting golgi mesh')
golgi = RGB_to_int24(mesh2Image(path_to_corresponding_3D_image, golgi_mesh_file))
print('converting nucleus mesh')
nucleus = RGB_to_int24(mesh2Image(path_to_corresponding_3D_image, nucleus_mesh_file))



converting golgi mesh
converting nucleus mesh
Wall time: 10min 9s


In [3]:
%%time

import csv
# maybe need save them to speed things up
#mask = Img(os.path.join(filename_without_ext, 'handCorrection4.tif'))
mask = Img('handCorrection.tif')
mask = RGB_to_int24(mask)

raw_image = Img(path_to_corresponding_3D_image)
gogli_channel = 1
golgi_intensity = raw_image[..., gogli_channel]
nucleus_channel = 0
nucleus_intensity = raw_image[..., nucleus_channel]

if __DEBUG__:
    # check because channels are not the same channel nb as in IJ !!!
    plt.imshow(golgi_intensity[0])  # golgi channel
    plt.show()

    plt.imshow(nucleus_intensity[0])  # nucleus channel
    plt.show()

#tracks = Img(os.path.join(filename_without_ext, 'tracked_cells_resized.tif'))
tracks = Img('tracked_cells_resized.tif')
tracks = RGB_to_int24(tracks)

#local_id = Img(os.path.join(filename_without_ext, 'cell_identity.tif'))
local_id = Img('cell_identity.tif')
local_id = RGB_to_int24(local_id)


def apply_wshed_mask_to_further_segment_golgi_or_nucleus(mask, image_to_resegment):
    for iii, img in enumerate(image_to_resegment):
        img[mask != 0] = 0

    seed = 1  # always start tracking with same seed to have roughly the same color
    assigned_ids = get_forbidden_colors_int24()

    # recolor separate regions
    # then do stuff for first encounter!!!
    lab = label(image_to_resegment, connectivity=1, background=0)
    rps = regionprops(lab)

    # need a whole set of random colors
    for rp in rps:
        new_col = get_unique_random_color_int24(forbidden_colors=assigned_ids, assign_new_col_to_forbidden=True)
        image_to_resegment[rp.coords[:, 0], rp.coords[:, 1], rp.coords[:, 2]] = new_col  # nb should be that for 3D

    return image_to_resegment


# further resegment gogli and nucleus using cell outlines/watershed mask
golgi = apply_wshed_mask_to_further_segment_golgi_or_nucleus(mask, golgi)
nucleus = apply_wshed_mask_to_further_segment_golgi_or_nucleus(mask, nucleus)

lab_golgi = label(golgi, connectivity=2, background=0)
lab_nuc = label(nucleus, connectivity=2, background=0)

rps_golgi = regionprops(lab_golgi, intensity_image=golgi_intensity)
rps_nuc = regionprops(lab_nuc, intensity_image=nucleus_intensity)

mapping_global_id_golgi_centoids = {}

for rps in rps_golgi:
    id = golgi[rps.coords[0][0], rps.coords[0][1], rps.coords[0][2]]
    mapping_global_id_golgi_centoids[id] = [rps.centroid, rps.weighted_centroid]

mapping_global_id_nuc_centoids = {}

for rps in rps_nuc:
    id = nucleus[rps.coords[0][0], rps.coords[0][1], rps.coords[0][2]]
    mapping_global_id_nuc_centoids[id] = [rps.centroid, rps.weighted_centroid]

    # nucleus[int(rps.centroid[0]), int(rps.centroid[1]), int(rps.centroid[2])] = 0xFFFFFF
    # try:
    #     nucleus[int(rps.weighted_centroid[0]), int(rps.weighted_centroid[1]), int(rps.weighted_centroid[2])] = 0xFFFF00
    # except:
    #     # traceback.print_exc()
    #     print('error in weighted centroid, most likely a black image')
    # raw_image[int(rps.centroid[0]), int(rps.centroid[1]), int(rps.centroid[2]),1]=0 # plot it on orig or not ??? --> if so how/which color

# just to visualize the centoids on the real image
# Img(int24_to_RGB(nucleus), dimensions='dhwc').save('/D/Sample_images/clara_tests_3D_mesh/output/nuc_colored_centroids.tif')

cell_n_first_golgi = {}
cell_n_first_nuc = {}
cell_n_local_id = {}

lab_tracks = label(tracks, connectivity=2, background=0xFFFFFF)
rps_tracks = regionprops(lab_tracks)

for rps in rps_tracks:

    cell_id = tracks[rps.coords[0][0], rps.coords[0][1]]
    cell_n_first_golgi[cell_id] = None
    cell_n_first_nuc[cell_id] = None
    cell_n_local_id[cell_id] = local_id[rps.coords[0][0], rps.coords[0][1]]

    for img in golgi:

        most_frequent_segmentation_mask, count = np.unique(img[rps.coords[:, 0], rps.coords[:, 1]], return_counts=True)
        count_sort_ind = np.argsort(-count)
        most_frequent_segmentation_mask = most_frequent_segmentation_mask[count_sort_ind]
        most_frequent_segmentation_mask = most_frequent_segmentation_mask.tolist()
        if len(most_frequent_segmentation_mask) == 1 and most_frequent_segmentation_mask[0] == 0:
            # print('unique ids', np.unique(img[rps.coords[:, 0], rps.coords[:, 1]]))  # ça marche
            continue
        else:
            # first encounter biggest golgi
            # get the first non 0 value this is the id I want
            most_frequent_segmentation_mask = set(most_frequent_segmentation_mask)
            if 0 in most_frequent_segmentation_mask:
                most_frequent_segmentation_mask.remove(0)
            most_frequent = list(most_frequent_segmentation_mask)[0]
            cell_n_first_golgi[cell_id] = most_frequent
            # print('unique ids',cell_id, cell_n_first_golgi[cell_id])  # ça marche
            break

    for img in nucleus:
        most_frequent_segmentation_mask, count = np.unique(img[rps.coords[:, 0], rps.coords[:, 1]], return_counts=True)
        # print(most_frequent_segmentation_mask, count)
        count_sort_ind = np.argsort(-count)
        most_frequent_segmentation_mask = most_frequent_segmentation_mask[count_sort_ind]
        most_frequent_segmentation_mask = most_frequent_segmentation_mask.tolist()

        if len(most_frequent_segmentation_mask) == 1 and most_frequent_segmentation_mask[0] == 0:
            continue
        else:
            # first encounter biggest golgi
            # get the first non 0 value this is the id I want
            most_frequent_segmentation_mask = set(most_frequent_segmentation_mask)
            if 0 in most_frequent_segmentation_mask:
                most_frequent_segmentation_mask.remove(0)
            most_frequent = list(most_frequent_segmentation_mask)[0]
            cell_n_first_nuc[cell_id] = most_frequent
            break

# store nucleus/golgi 3D centroid position in a csv file
header = ['local_id', 'track_id', 'nuc_centroid_z', 'nuc_centroid_y', 'nuc_centroid_x', 'nuc_weighted_centroid_z',
          'nuc_weighted_centroid_y', 'nuc_weighted_centroid_x', 'golgi_centroid_z', 'golgi_centroid_y',
          'golgi_centroid_x', 'golgi_weighted_centroid_z', 'golgi_weighted_centroid_y', 'golgi_weighted_centroid_x']
data = []
data.append(header)
for id in cell_n_first_golgi.keys():
    golgi_id = cell_n_first_golgi[id]
    nuc_id = cell_n_first_nuc[id]
    local_id = cell_n_local_id[id]

    cur_row = [local_id, id]
    if nuc_id is not None:
        cur_row.extend(list(mapping_global_id_nuc_centoids[nuc_id][0]))
        cur_row.extend(list(mapping_global_id_nuc_centoids[nuc_id][1]))
    else:
        cur_row.extend(['n.a.', 'n.a.', 'n.a.', 'n.a.', 'n.a.', 'n.a.'])  # could replace by other things

    if golgi_id is not None:
        cur_row.extend(list(mapping_global_id_golgi_centoids[golgi_id][0]))
        cur_row.extend(list(mapping_global_id_golgi_centoids[golgi_id][1]))
    else:
        cur_row.extend(['n.a.', 'n.a.', 'n.a.', 'n.a.', 'n.a.', 'n.a.'])

    # code to draw the 2D projection corresponding to the 3D nuc to golgi connection
    # code below can be commented
    if golgi_id is not None and nuc_id is not None:  # still need print if they are not good
        rr, cc = draw.line(int(mapping_global_id_nuc_centoids[nuc_id][0][1]),
                           int(mapping_global_id_nuc_centoids[nuc_id][0][2]),
                           int(mapping_global_id_golgi_centoids[golgi_id][0][1]),
                           int(mapping_global_id_golgi_centoids[golgi_id][0][2]))
        tracks[rr, cc] = 0xFFFFFF

    data.append(cur_row)

file = open(os.path.join('E2_nuc_to_golgi_3d.csv'), 'w', newline='')
with file:
    writer = csv.writer(file, delimiter='\t')# make csv file tab separated --> not a real csv file
    writer.writerows(data)

# save or not, up to you...
# save 2D version of golgi
Img(int24_to_RGB(tracks), dimensions='hwc').save(    os.path.join(filename_without_ext, '2D_version_of_3D_nuc_to_golgi_vector.tif'))
Img(int24_to_RGB(nucleus), dimensions='dhwc').save(
    os.path.join(filename_without_ext, 'whsed_resplit_nuclei_labels.tif'))
Img(int24_to_RGB(golgi), dimensions='dhwc').save(os.path.join(filename_without_ext, 'whsed_resplit_golgi_labels.tif'))


C:\Users\guijarro\AppData\Roaming\Python\Python37\site-packages\skimage\measure\_regionprops.py:480: RuntimeWarning: invalid value encountered in true_divide
  M[(0,) * self._ndim])


Wall time: 10.3 s
